
Dans ce notebook, nous allons nous occuper du preprocess c'est à dire du nettoyage et de la mise en forme des données en vue de la création du modèle. Cette partie n'est pas documentée et ne contient que le code qui sert proprement à la modification du dataframe pour le preprocess.

In [7]:
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [8]:
df = pd.read_csv("/home/onyxia/PY-Music-Genre-Classifier/src/spotify_tracks.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/home/onyxia/PY-Music-Genre-Classifier/src/spotify_tracks.csv'

In [ ]:
df.drop(columns=["track_name","track_id","track_artist","track_album_id","track_album_name","track_album_release_date",
       "playlist_name", "playlist_id"], inplace = True)

In [ ]:
def winsorize_outliers(df, columns, threshold=3):
    """
    Winsorise les valeurs aberrantes (outliers) dans les colonnes continues d'un DataFrame,
    sans modifier le DataFrame original.

    Parameters:
    - df (pd.DataFrame): DataFrame contenant les colonnes continues.
    - columns (list): Liste des colonnes à traiter.
    - threshold (float): Seuil z-score pour définir les outliers.

    Returns:
    - pd.DataFrame: Nouveau DataFrame avec les colonnes ajustées.
    """
    # Créer une copie du DataFrame pour ne pas modifier l'original
    df_copy = df.copy()

    for column in columns:
        z_scores = zscore(df_copy[column])
        
        # Calcul des limites
        lower_bound = df_copy[column][z_scores > -threshold].min()
        upper_bound = df_copy[column][z_scores < threshold].max()
        
        # Winsorisation
        df_copy[column] = np.clip(df_copy[column], lower_bound, upper_bound)
        
    return df_copy

df_winsorized = winsorize_outliers(df, ['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'tempo','duration_ms'], 3)

In [ ]:
df = df_winsorized

In [ ]:
columns_to_standardize = ['track_popularity', 'danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
        'duration_ms']

scaler = StandardScaler()
df_standardized = df.copy()
df_standardized[columns_to_standardize] = scaler.fit_transform(df_standardized[columns_to_standardize])

In [ ]:
df = df_standardized

In [ ]:
predictors = ['track_popularity', 'speechiness', 'instrumentalness', 'energy', 'danceability', 'acousticness', 'tempo', 
              'duration_ms', 'loudness', 'key', 'mode']

In [ ]:
def impute_with_random_forest(df, target_column, predictors, random_state=42):
    """
    Impute les valeurs manquantes d'une colonne avec un Random Forest Regressor.

    Arguments :
    - df : DataFrame pandas
    - target_column : Nom de la colonne cible à imputer
    - predictors : Liste des colonnes à utiliser comme prédicteurs
    - random_state : sert à la reproductibilité

    Retourne :
    - DataFrame avec les valeurs imputées pour la colonne cible
    """
    df_rf = df.copy()
    
    train_data = df_rf[df_rf[target_column].notna()]
    test_data = df_rf[df_rf[target_column].isna()]

    if test_data.empty:
        return df_rf

    X_train = train_data[predictors]
    y_train = train_data[target_column]

    X_test = test_data[predictors]

    rf_model = RandomForestRegressor(n_estimators=100, random_state=random_state)
    rf_model.fit(X_train, y_train)

    predicted_values = rf_model.predict(X_test)

    df_rf.loc[df[target_column].isna(), target_column] = predicted_values

    return df_rf


df_intermediary = impute_with_random_forest(df, target_column='liveness', predictors=predictors)

df_rf = impute_with_random_forest(df_intermediary, target_column='valence', predictors=predictors)


In [ ]:
df = df_rf

In [ ]:
subgenre_proportions = df['playlist_subgenre'].value_counts(normalize=True)
df['playlist_subgenre_encoded'] = df['playlist_subgenre'].map(subgenre_proportions)
df = df.drop(columns=['playlist_subgenre'])

,track_popularity,playlist_genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,playlist_subgenre_encoded
0,0.941531,pop,0.643106,1.205761,6,1.417813,1,-0.493390,-0.334394,-0.386352,-0.808248,0.030257,0.043509,-0.530427,0.039533
1,0.981557,pop,0.490988,0.644866,11,0.601505,1,-0.711650,-0.471831,-0.365805,1.082508,0.781581,-0.778346,-1.092531,0.039533
2,1.101635,pop,0.138349,1.289062,1,1.138835,0,-0.328136,-0.439329,-0.386238,-0.518509,0.438119,0.116963,-0.847509,0.039533
3,0.701374,pop,0.435672,1.283509,7,1.017875,1,-0.039202,-0.674736,-0.386306,0.090785,-1.004425,0.040529,-0.979023,0.039533
4,1.061609,pop,-0.034513,0.744828,1,0.705335,1,-0.726201,-0.435150,-0.386352,-0.691575,0.918967,0.115771,-0.630107,0.039533
